In [ ]:
%%html
<link rel="stylesheet" href="cc-jupyter.css"/>

# Functional Programming

## 1. Immutable Data

The most straightforward way of avoiding side effects is to use immutable data structures.
Use them as often as possible.
The namedtuple is a useful replacement for the builtin dict.

In [ ]:
from collections import namedtuple

# create an immutable dict-like class
faux_dict = namedtuple('faux_dict', ['name', 'age'])
person = faux_dict(name='Max Mustermann', age=44)

access values through dot-notation

In [ ]:
print(person.name)
print(person.age)

due to immutability, updating a value requires that you copy the entire object.

In [ ]:
print(person._replace(age=87))

# the original object stays unchanged
print(person)

## 2. Trade methods for functions

Try to treat mutable objects (e.g. lists) just like immutable objects.
For this reason, it’s not *Ok* to `sort` a list in-place in FP.

In [ ]:
titles = ['Star Wars', '2001', 'Solaris']
print(titles)
titles.sort()
print(titles)   # mutated(iii!)

It’s better to use the `sorted` function which returns a copy of the list and avoids mutation.

In [ ]:
titles = ['Star Wars', '2001', 'Solaris']
print(sorted(titles, key=len))
print(titles)

Strings are already immutable in Python but functions are still more flexible to use than methods in FP. Each string method is available as a complementary function under the str/unicode class.

In [ ]:
titles = 'Star Wars, 2001, Solaris'
print(titles.split(','))
print(str.split(titles, ','))

## 3. map, filter, reduce

These three functions are all staple FP functions found in Python.
They work on any iterable and always accept a function as the first parameter.

In [ ]:
titles = ['    Star Wars ', '2001    ', '  Solaris']
list(map(str.strip, titles))

In Python-land, map and filter are implemented as list comprehension.
So the following expression might be a bit more pythonic:

In [ ]:
[str.strip(title) for title in titles]

`map` is lazy and therefore the same as the complementary generator expression.

In [ ]:
print((str.strip(title) for title in titles))
print(map(str.strip, titles))

# iterate over a generator/map to see its content
print(list(map(str.strip, titles)))  

`filter` is in the same sense nothing more than a list comprehension + a conditional statement.

In [ ]:
titles = ['Star Wars', '2001', 'Solaris']

def is_one_word(string):
    return len(string.split()) == 1

print(list(filter(is_one_word, titles)))
print([title for title in titles if is_one_word(title)])

## 4. Currying

-   [Curried](http://toolz.readthedocs.org/en/latest/curry.html)
    functions partially evaluates if they do not receive enough arguments to compute a result.

-   … is normally accomplished using functools.partial.

In [ ]:
%pip install -q toolz

[ipython magic
commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

In [ ]:
from toolz import curry
@curry
def multiply(first_factor, second_factor):
  return first_factor * second_factor

double = multiply(2)
double(21)

## A simple stream reader

implement a simple reader for a Tab-delimited stream (or file object).

It will be lazy (memory efficient), linear (parallelizable), and easy to comprehend.

In [ ]:
# The input is really any iterable, 
# whether a file object, list, 
# or generator that spits out lines of text.
stream = [
  '#time\tevent\n',
  '003 min\t"The Daily Show" starts       \n',
  '013 min\t"Star Wars IV" airs on TV     \n',
  '105 min\t"Monty Python`s Flying Circus" starts\n',
  '149 min\tFrogs begin to fall from the sky\n',
]

We need to setup a few layer of filters first.

In [ ]:
from toolz import curry
@curry
def is_comment_line(line, prefix='#'):
  """Check if a string starts with a specified prefix."""
  return str.startswith(line, prefix)

print(is_comment_line('foo'))
print(is_comment_line('#bar'))

combine to fully working pipeline…

In [ ]:
from toolz import pipe, partial
from toolz.curried import complement, map, filter
sequence = pipe(
  stream,
  filter(complement(is_comment_line)),   # filter out comments
  map(str.rstrip),                       # strip invisible chars
  map(partial(str.split, sep='\t')),     # split lines
)

list(sequence)

Lazy, linear, and without side effects. Beautiful.